In [1]:
import pandas as pd
from pandas.tseries.holiday import *
cal = USFederalHolidayCalendar()

train = pd.read_csv("sample.csv")
test = pd.read_csv("test.csv")

In [2]:
train = train.dropna()

train.drop(['key','passenger_count'],inplace=True,axis=1)

train.drop(train.index[(train.pickup_longitude < -80) | (train.pickup_longitude > -65) | (train.pickup_latitude < 30) | (train.pickup_latitude > 50)],inplace=True)

train.drop(train.index[(train.dropoff_longitude < -80) | (train.dropoff_longitude > -65) | (train.dropoff_latitude < 30) | (train.dropoff_latitude > 50)],inplace=True)

train.drop(train.index[train.fare_amount <= 2.5],inplace=True)

train['pickup_datetime']  = pd.to_datetime(train['pickup_datetime'])

train.reset_index(drop=True,inplace=True)

In [3]:
train['day_of_week'] = train['pickup_datetime'].dt.dayofweek
train['year'] = train['pickup_datetime'].dt.year
train['month'] = train['pickup_datetime'].dt.month
train['hour'] = train['pickup_datetime'].dt.hour

holidays = cal.holidays(start=train['pickup_datetime'].dt.date.min(),
                        end=train['pickup_datetime'].dt.date.max())
train['holiday'] = train['pickup_datetime'].dt.date.astype('datetime64').isin(holidays).astype('uint8')

train_target = train['fare_amount']

train.drop(['fare_amount','pickup_datetime'],inplace=True,axis=1)

In [4]:
test_keys = test['key']
test.drop(['key','passenger_count'],inplace=True,axis=1)
test['pickup_datetime']  = pd.to_datetime(test['pickup_datetime'])

In [5]:
test['day_of_week'] = test['pickup_datetime'].dt.dayofweek
test['year'] = test['pickup_datetime'].dt.year
test['month'] = test['pickup_datetime'].dt.month
test['hour'] = test['pickup_datetime'].dt.hour

holidays = cal.holidays(start=test['pickup_datetime'].dt.date.min(),
                        end=test['pickup_datetime'].dt.date.max())
test['holiday'] = test['pickup_datetime'].dt.date.astype('datetime64').isin(holidays).astype('uint8')

test.drop('pickup_datetime',inplace=True,axis=1)

In [6]:
%%time
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(n_estimators = 500, max_depth=6)
reg.fit(train, train_target)

CPU times: user 29min 8s, sys: 764 ms, total: 29min 8s
Wall time: 29min 7s


GradientBoostingRegressor(max_depth=6, n_estimators=500)

In [7]:
preds = reg.predict(test)
submission = pd.DataFrame({'key': test_keys, 'fare_amount':preds.round(2)})
submission.to_csv('sample_submission.csv',index=False)